In [1]:
# Libreries
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from googleapiclient.discovery import build
from langchain_groq import ChatGroq


In [2]:
# APIS



In [3]:
from dotenv import load_dotenv

load_dotenv()

youtude_api = os.getenv("YOUTUBE_API_KEY")
groq_api = os.getenv("GROQ_API_KEY")
# print(groq_api)

In [4]:
def search_youtube_videos(query, max_results=5):
    youtube = build("youtube", "v3", developerKey=youtude_api)
    response = youtube.search().list(
        q=query,
        part="snippet",
        maxResults=max_results,
        type=["video", "playlist"],
    ).execute()

    # print(response)

    videos = []
    for item in response.get("items", []):
        if item["id"]["kind"] != "youtube#video":
            title = item["snippet"]["title"]
            description = item["snippet"]["description"]
            playlist_id = item["id"]["playlistId"]
            url = f"https://www.youtube.com/playlist?list={playlist_id}"
            videos.append({
                "title": title,
                "description": description,
                "url": url
            })
        else:
            title = item["snippet"]["title"]
            description = item["snippet"]["description"]
            video_id = item["id"]["videoId"]
            url = f"https://www.youtube.com/watch?v={video_id}"
            videos.append({
                "title": title,
                "description": description,
                "url": url
            })
    return videos

In [5]:
# SEARCH_QUERY = input("Enter what do you want to search: ")
SEARCH_QUERY = "python course"
videos = search_youtube_videos(SEARCH_QUERY)

print(f'\nvideos: {videos}')


videos: [{'title': 'Python Full Course for Beginners [2025]', 'description': 'Master Python from scratch No fluff—just clear, practical coding skills to kickstart your journey! ❤️ Join this channel to get ...', 'url': 'https://www.youtube.com/watch?v=K5KVEU3aaeQ'}, {'title': 'Mastering Python - تعلم بايثون', 'description': 'Mastering Python in Arabic And Learn Info About All Paths.', 'url': 'https://www.youtube.com/playlist?list=PLDoPjvoNmBAyE_gei5d18qkfIe-Z8mocs'}, {'title': 'Python Full Course for free 🐍 (2024)', 'description': "python #tutorial #beginners Python tutorial for beginners' full course 2024 *Learn Python in 1 HOUR* ...", 'url': 'https://www.youtube.com/watch?v=ix9cRaBkVe0'}, {'title': 'Python Full Course for Beginners', 'description': 'Learn Python for AI, machine learning, and web development with this beginner-friendly course! Get 6 months of PyCharm ...', 'url': 'https://www.youtube.com/watch?v=_uQrJ0TkZlc'}, {'title': 'Learn Python - Full Course for Beginners [Tutor

In [6]:
def formate_videos_metadata(videos):
    metadata = ""
    
    for idx, vid in enumerate(videos, start = 1):
        metadata += f"{idx}. Title: {vid['title']}\nDescription: {vid['description']}\n Link: {vid['url']}\n\n"
    
    return metadata

In [7]:
videos_metadata = formate_videos_metadata(videos)
print(f'\n\nvideos_metadata: {videos_metadata}')



videos_metadata: 1. Title: Python Full Course for Beginners [2025]
Description: Master Python from scratch No fluff—just clear, practical coding skills to kickstart your journey! ❤️ Join this channel to get ...
 Link: https://www.youtube.com/watch?v=K5KVEU3aaeQ

2. Title: Mastering Python - تعلم بايثون
Description: Mastering Python in Arabic And Learn Info About All Paths.
 Link: https://www.youtube.com/playlist?list=PLDoPjvoNmBAyE_gei5d18qkfIe-Z8mocs

3. Title: Python Full Course for free 🐍 (2024)
Description: python #tutorial #beginners Python tutorial for beginners' full course 2024 *Learn Python in 1 HOUR* ...
 Link: https://www.youtube.com/watch?v=ix9cRaBkVe0

4. Title: Python Full Course for Beginners
Description: Learn Python for AI, machine learning, and web development with this beginner-friendly course! Get 6 months of PyCharm ...
 Link: https://www.youtube.com/watch?v=_uQrJ0TkZlc

5. Title: Learn Python - Full Course for Beginners [Tutorial]
Description: This course will g

In [8]:
def prompt(user_query, video_metadata_list):
    prompt_template = PromptTemplate(
        input_variables=["user_query", "video_metadata_list"],
        template= """
You are an intelligent assistant helping users find the best YouTube videos.

User message: "{user_query}"

From the following videos, recommend the top 3 that are most relevant and useful:

{video_metadata_list}

Give reasult including the title and link only in the following format:
1. Title: video title, Link: video link
2. Title: video title, Link: video link
3. Title: video title, Link: video link

"""
)
    # Give reasons for each choice and include the title and link only.
    # "You are a Youtube video search engine. You have to answer the user query based on the metadata of the videos.\n\nUser Query: {user_query}\n\nMetadata:\n{metadata}\n\nAnswer:"
    return prompt_template

In [9]:
prompt = prompt(SEARCH_QUERY, videos_metadata)
print(f'\n\nprompt: {prompt}')



prompt: input_variables=['user_query', 'video_metadata_list'] template='\nYou are an intelligent assistant helping users find the best YouTube videos.\n\nUser message: "{user_query}"\n\nFrom the following videos, recommend the top 3 that are most relevant and useful:\n\n{video_metadata_list}\n\nGive reasult including the title and link only in the following format:\n1. Title: video title, Link: video link\n2. Title: video title, Link: video link\n3. Title: video title, Link: video link\n\n'


In [10]:
MAX_RESULTS = 5

def recommend_videos_with_llm(user_query, prompt_template):
    videos = search_youtube_videos(user_query, MAX_RESULTS)
    formatted_list = formate_videos_metadata(videos)

    llm = ChatGroq(
        model="llama3-8b-8192",
        temperature=0.5
        # ,api_key = GROQ_API_KEY
    )

    model = LLMChain(llm=llm, prompt=prompt_template)
    response = model.invoke({
        "user_query": user_query,
        "video_metadata_list": formatted_list
    })

    return response["text"]

In [11]:
result = recommend_videos_with_llm(SEARCH_QUERY, prompt)
print(result)

C:\Users\suhai\AppData\Local\Temp\ipykernel_14688\120397845.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  model = LLMChain(llm=llm, prompt=prompt_template)


Based on the user's query "python course", I recommend the top 3 most relevant and useful videos:

1. Title: Python Full Course for Beginners [2025], Link: https://www.youtube.com/watch?v=K5KVEU3aaeQ
2. Title: Learn Python - Full Course for Beginners [Tutorial], Link: https://www.youtube.com/watch?v=rfscVS0vtbw
3. Title: Python Full Course for Beginners, Link: https://www.youtube.com/watch?v=_uQrJ0TkZlc
